In [3]:
using Random
rng = Random.default_rng()

TaskLocalRNG()

In [4]:
Threads.nthreads()

8

---
# Bulid an ANN model

---

In [2]:
using Flux
nn_model = Flux.Chain(
    Flux.Dense(1, 32, tanh), 
    Flux.Dense(32, 32, tanh), 
    Flux.Dense(32, 1)
    )

Chain(
  Dense(1 => 32, tanh),                 # 64 parameters
  Dense(32 => 32, tanh),                # 1_056 parameters
  Dense(32 => 1),                       # 33 parameters
)                   # Total: 6 arrays, 1_153 parameters, 4.879 KiB.

In [3]:
nn_param, nn_rebulid = Flux.destructure(nn_model)
# nn_param
# nn_rebulid(nn_param)

(Float32[0.2376694, -0.11739163, 0.2498732, 0.098599, 0.14856294, -0.31346285, 0.096432835, -0.33599928, 0.3111586, -0.26225471  …  0.31898645, -0.035142682, -0.028881673, -0.17904513, 0.037371162, 0.024970887, 0.10860127, -0.371204, -0.42222318, 0.0], Restructure(Chain, ..., 1153))

---
# Define a OED Problem
---

In [4]:
using DifferentialEquations

mumax(T) = -0.00266071 .* T .^ 2 .+ 0.18037857 .* T .- 2.86700000
xmax(T) = 0.76071429 .* T .^ 2 .- 55.69857143 .* T .+ 1058.18000000
pmax(T) = 0.57678571 .* T .^ 2 .- 42.06642857 .* T .+ 838.23200000
ypx(T) = -0.02071429 .* T .^ 2 .+ 1.90357143 .* T .- 26.44400000
yx(T) = -0.00016071 .* T .^ 2 .+ 0.00917857 .* T .- 0.09160000

function ethanol(dy, y, p, t)
    h, cp, d, frec, v, ks, ki, m, n, mx, mp, Tsp = [-511 3.82 1060 31 59.66 4.1 4e-3 1.0 1.5 0.2 0.1 30.0]
    X = y[1]
    S = y[2]
    P = y[3]
    T = y[4]
    #* real(((1 - (P / pmax(T))) + 0im)^(n) * X) * ℯ^((-1) * ki * S)# *(ℯ)^(-ki*S) * ((1-(X/xmax(T)))^m) * (((1-(P/pmax(T)))^n)*X)
    dy[1] = mumax(T) * (S / (S + ks)) * real((1 - (X / xmax(T)) + 0im)^m) 
    dy[2] = -((dy[1] / yx(T)) + (mx * X))
    dy[3] = (ypx(T) * dy[1]) + (mp * X)
    dy[4] = (h / (d * cp)) * dy[2] - (frec / v) * nn_rebulid(p)([t])[1]
end

y0 = [2.75f0, 150.0f0, 0.0f0, 30.0f0]
tspan = (0.0f0, 33.0f0)
ts = Float32.(collect(0.0:0.01:tspan[2]))
prob = DifferentialEquations.ODEProblem(ethanol, y0, tspan, nn_param)
DifferentialEquations.solve(prob, Vern9(), abstol = 1e-10, reltol = 1e-10) 

retcode: Success
Interpolation: specialized 9th order lazy interpolation
t: 193639-element Vector{Float32}:
  0.0
  0.00027455817
  0.00043928565
  0.0006009359
  0.0007532079
  0.00088613987
  0.0010066313
  0.001130778
  0.0012357149
  0.0013633197
  ⋮
 32.992382
 32.993378
 32.994408
 32.995403
 32.99644
 32.997463
 32.9985
 32.999516
 33.0
u: 193639-element Vector{Vector{Float32}}:
 [2.75, 150.0, 0.0, 30.0]
 [2.7500384, 149.99887, 0.00053806877, 30.000143]
 [2.7500615, 149.99818, 0.0008609019, 30.000229]
 [2.7500842, 149.99751, 0.0011777089, 30.000313]
 [2.7501056, 149.99689, 0.0014761402, 30.000393]
 [2.7501242, 149.99634, 0.0017366712, 30.000462]
 [2.7501411, 149.99583, 0.0019728227, 30.000525]
 [2.7501585, 149.99532, 0.002216141, 30.00059]
 [2.7501733, 149.99489, 0.0024218115, 30.000645]
 [2.7501912, 149.99435, 0.0026719125, 30.000711]
 ⋮
 [4.2304163, 112.72332, 34.185196, 42.82665]
 [4.2303963, 112.725426, 34.185276, 42.82665]
 [4.230376, 112.72761, 34.18536, 42.826645]
 [4.230

---
# Define Optimization Problem
---

In [6]:
using SciMLSensitivity
function predict_adjoint(θ)
    Array(
        SciMLSensitivity.solve(prob, Vern9(), p = θ, saveat = ts,
        sensealg = InterpolatingAdjoint())
        )
end

using Statistics
function loss_adjoint(θ)
    x = predict_adjoint(θ)
    mean(abs2, 30.0 .- x[4, :]) +
    mean(abs2, [first(nn_rebulid(θ)([t])) for t in ts]) / 10
end

loss_nn_param = loss_adjoint(nn_param)

113.40453501594482

In [7]:
using Plots
callback = function (param, loss; doplot = false)
    println(loss)

    if doplot
        p = plot(solve(remake(prob, p = param), Tsit5(), saveat = 0.01), ylim = (-6, 6), lw = 3)
        plot!(p, ts, [first(nn_rebulid(param)([t])) for t in ts], label = "u(t)", lw = 3)
        display(p)
    end

    return false
end

# Display the ODE with the current parameter values.
callback(nn_param, loss_nn_param)

113.40453501594482


false

In [8]:
using Optimization, OptimizationFlux
# Setup and run the optimization
# loss1 = loss_adjoint(nn_param)
adtype = Optimization.AutoZygote()
optf = Optimization.OptimizationFunction((x, p) -> loss_adjoint(x), adtype)

(::OptimizationFunction{true, AutoZygote, var"#23#24", Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED_NO_TIME), Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing}) (generic function with 1 method)

In [9]:
optprob = Optimization.OptimizationProblem(optf, nn_param)

OptimizationProblem. In-place: true
u0: 1153-element Vector{Float32}:
  0.2376694
 -0.11739163
  0.2498732
  0.098599
  0.14856294
 -0.31346285
  0.096432835
 -0.33599928
  0.3111586
 -0.26225471
  ⋮
 -0.035142682
 -0.028881673
 -0.17904513
  0.037371162
  0.024970887
  0.10860127
 -0.371204
 -0.42222318
  0.0

In [10]:
res1 = Optimization.solve(optprob, ADAM(0.005), callback = callback, maxiters = 100)

┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity /Users/zewen/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21
Enzyme compilation failed due to illegal type analysis.


Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__10134({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* addrspace(11)*, !dbg !3399
  %5 = getelementptr inbounds {} addrspace(10)*, {} addrspace(10)* addrspace(11)* %4, i64 3, !dbg !3399
  %6 = bitcast {} addrspace(10)* addrspace(11)* %5 to i64 addrspace(11)*, !dbg !3399
  %7 = load i64, i64 addrspace(11)* %6, align 8, !dbg !3399, !tbaa !861, !range !863, !invariant.lo


Illegal updateAnalysis prev:{[-1]:Pointer, [-1,0]:Integer, [-1,1]:Integer, [-1,2]:Integer, [-1,3]:Integer, [-1,4]:Integer, [-1,5]:Integer, [-1,6]:Integer, [-1,7]:Integer} new: {[-1]:Pointer, [-1,0]:Float@double}
val:   %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104 origin=  %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104

Caused by:
Stacktrace:
 [1] Float64
   @ ./float.jl:159
 [2] convert
   @ ./number.jl:7
 [3] setindex!
   @ ./array.jl:971
 [4] hvcat_fill!
   @ ./abstractarray.jl:2135



113.40453501594482

┌ Warning: ReverseDiffVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity /Users/zewen/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:61
ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float32, Nothing} to Int64 is not defined. Please use `ReverseDiff.value` instead.

...

Illegal updateAnalysis prev:{[-1]:Pointer, [-1,0]:Integer, [-1,1]:Integer, [-1,2]:Integer, [-1,3]:Integer, [-1,4]:Integer, [-1,5]:Integer, [-1,6]:Integer, [-1,7]:Integer} new: {[-1]:Pointer, [-1,0]:Float@double}
val:   %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104 origin=  %36 = bitcast {} addrspace(10)* %27 to i64 addrspace(10)*, !dbg !104

Caused by:
Stacktrace:
 [1] Float64
   @ ./float.jl:159
 [2] convert
   @ ./number.jl:7
 [3] setindex!
   @ ./array.jl:971
 [4] hvcat_fill!
   @ ./abstractarray.jl:2135
┌ Warning: ReverseDiffVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity /Users/zewen/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:61
ArgumentError: Converting an instance of ReverseDiff.TrackedReal{Float64, Float32, Nothing} to Int64 is not defined. Please use `ReverseDiff.value` instead.┌ Warning: Automatic dt set the s

┌ Warning: dt(3.8146973e-6) <= dtmin(3.8146973e-6) at t=6.7866306, and step error estimate = 0.9168464. Aborting. There is either an error in your model specification or the true solution is unstable.
└ @ SciMLBase /Users/zewen/.julia/packages/SciMLBase/kTUaf/src/integrator_interface.jl:599
┌ Warning: EnzymeVJP tried and failed in the automated AD choice algorithm with the following error. (To turn off this printing, add `verbose = false` to the `solve` call)
└ @ SciMLSensitivity /Users/zewen/.julia/packages/SciMLSensitivity/NhfkF/src/concrete_solve.jl:21


Excessive output truncated after 531691 bytes.

Enzyme compilation failed due to illegal type analysis.
Current scope: 
; Function Attrs: mustprogress willreturn
define internal fastcc noundef nonnull align 16 dereferenceable(40) {} addrspace(10)* @preprocess_julia_hvcat_fill__15180({} addrspace(10)* noundef nonnull readonly returned align 16 dereferenceable(40) %0, { i64, double, i64, i64, double, double, double, double, double, double, double, double } addrspace(11)* nocapture nofree noundef nonnull readonly align 8 dereferenceable(96) %1) unnamed_addr #41 !dbg !3398 {
top:
  %2 = call {}*** @julia.get_pgcstack() #42
  %3 = bitcast {} addrspace(10)* %0 to {} addrspace(10)* addrspace(10)*, !dbg !3399
  %4 = addrspacecast {} addrspace(10)* addrspace(10)* %3 to {} addrspace(10)* addrspace(11)*, !dbg !3399
  %5 = getelementptr inbounds {} addrspace(10)*, {} addrspace(10)* addrspace(11)* %4, i64 3, !dbg !3399
  %6 = bitcast {} addrspace(10)* addrspace(11)* %5 to i64 addrspace(11)*, !dbg !3399
  %7 = load i64, i64 addrspace(11)* %6, ali

u: 1153-element Vector{Float32}:
  0.22784099
 -0.10759628
  0.24005733
  0.0889183
  0.1447865
 -0.3036161
  0.0867515
 -0.345832
  0.30958566
 -0.27192757
  ⋮
 -0.025406273
 -0.018965902
 -0.16907763
  0.046855703
  0.015160136
  0.118211575
 -0.36129722
 -0.41246337
  0.009684246

In [11]:
using OptimizationNLopt
optprob2 = Optimization.OptimizationProblem(optf, res1.u)
res2 = Optimization.solve(optprob2, NLopt.LD_LBFGS(), maxiters = 100)

In [ ]:
function loss_adjoint(θ)
    x = predict_adjoint(θ)
    mean(abs2, 30.0 .- x[4, :]) +
    mean(abs2, [first(nn_rebulid(θ)([t])) for t in ts])
end
optf3 = Optimization.OptimizationFunction((x, p) -> loss_adjoint(x), adtype)

optprob3 = Optimization.OptimizationProblem(optf3, res2.u)
res3 = Optimization.solve(optprob3, NLopt.LD_LBFGS(), maxiters = 100)

In [ ]:
l = loss_adjoint(res3.u)
callback(res3.u, l)
p = plot(solve(remake(prob, p = res3.u), Tsit5(), saveat = 0.01), ylim = (-6, 6), lw = 3)
plot!(p, ts, [first(ann_rebulid(res3.u)([t])) for t in ts], label = "u(t)", lw = 3)